In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Load the dataset
path = r'/content/drive/MyDrive/Gen_AI/clean1.csv'
df = pd.read_csv(path)

In [ ]:
#!pip install langchain

In [ ]:
df.head(2)

,num,name,content
0,9180533,the.message.(1976).eng.1cd,Watch any video online with OpenSUBTITLES Free...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,Ah Theres Princess Dawn and Terry with the Blo...


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_documents_langchain(df, chunk_size=512, chunk_overlap=100):
    """Chunks documents using Langchain and returns a new DataFrame with metadata."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    chunks = []
    metadatas = []
    for index, row in df.iterrows():
        # Convert 'content' to string before splitting
        content = str(row["content"])
        doc_chunks = text_splitter.split_text(content)
        chunks.extend(doc_chunks)
        # Store subtitle_id and subtitle_name in the metadata
        metadatas.extend([{"original_index": index,
                          "subtitle_id": row["num"],
                          "subtitle_name": row["name"]}] * len(doc_chunks))

    chunk_df = pd.DataFrame({"chunk": chunks, "metadata": metadatas})
    return chunk_df

In [ ]:

# Example usage (assuming 'new_df' has a 'content' column):
chunked_df = chunk_documents_langchain(df)



In [ ]:
print(chunked_df.head(2))

                                               chunk  \
0  Watch any video online with OpenSUBTITLES Free...   
1  it like this when John the Baptist came to kin...   

                                            metadata  
0  {'original_index': 0, 'subtitle_id': 9180533, ...  
1  {'original_index': 0, 'subtitle_id': 9180533, ...  


In [ ]:
import os

path = "/content/drive/MyDrive/Gen_Ai/chuncked_cleaned_data"

# Check if the directory exists, create it if it doesn't
if not os.path.exists(path):
    os.makedirs(path)
    print(f"Directory created: {path}")
else:
    print(f"Directory already exists: {path}")

# Save the DataFrame as a Parquet file
try:
    chunked_df.to_parquet(f"{path}subtitles_extracted.parquet", engine="pyarrow", index=False)
    print("Successfully saved the dataset in:", path)
except Exception as e:
    print("Error:", e)

Directory created: /content/drive/MyDrive/Gen_Ai/chuncked_cleaned_data
Successfully saved the dataset in: /content/drive/MyDrive/Gen_Ai/chuncked_cleaned_data
